In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Preparing corpus

In [2]:
import os

path = 'proj-nlp-2024/data'
os.chdir(f'/content/drive/MyDrive/{path}')
os.getcwd()

'/content/drive/MyDrive/proj-nlp-2024/data'

Loading the trainig set from the json file

In [3]:
import json
train_path = os.path.join(os.getcwd(), 'beerqa_train_v1.0.json')
with open(train_path, 'r') as file:
    train_dataset = json.load(file)

In [ ]:
# train_dataset

In [34]:
from pandas.core.common import flatten

data_dict = train_dataset.get('data', {}) # Extracting the data dictionary
context_list = [list(flatten(entry.get('context', ''))) for entry in data_dict] # List of list of strings
# context_list = [[' '.join(string)] for string in context_list] # Making the title part of the paragraph

In [36]:
context_list[:3]

[['Edmund Burke',
  'In the 19th century, Burke was praised by both liberals and conservatives. Burke\'s friend Philip Francis wrote that Burke "was a man who truly & prophetically foresaw all the consequences which would rise from the adoption of the French principles", but because Burke wrote with so much passion, people were doubtful of his arguments. William Windham spoke from the same bench in the House of Commons as Burke had when he had separated from Fox and an observer said Windham spoke "like the ghost of Burke" when he made a speech against peace with France in 1801. William Hazlitt, a political opponent of Burke, regarded him as amongst his three favourite writers (the others being Junius and Rousseau) and made it "a test of the sense and candour of any one belonging to the opposite party, whether he allowed Burke to be a great man". William Wordsworth was originally a supporter of the French Revolution and attacked Burke in "A Letter to the Bishop of Llandaff" (1793), but 

Flattening the list of lists into a list of string

In [41]:
context_string = list(flatten(context_list))
context_string2 = context_string[1::2]

In [46]:
context_string2[3]

'Morales began work on his "indigenous autonomy" policy, which he launched in the eastern lowlands department on 3 August 2009. Bolivia was the first nation in the history of South America to affirm the right of indigenous people to self-government. Speaking in Santa Cruz Department, the President called it "a historic day for the peasant and indigenous movement", saying that, though he might make errors, he would "never betray the fight started by our ancestors and the fight of the Bolivian people." A vote on further autonomy for jurisdictions took place in December 2009, at the same time as general elections to office. The issue divided the country.'

Defining text preprocessing functions

In [ ]:
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# TODO: Remove roman numbers and specific date stuff!
# TODO: Remove \n

# make all text lowercase
def text_lowercase(text):
    return text.lower()
# remove numbers
def remove_numbers(text):
    result = re.sub(r'\d+', '', text)
    return result
# remove punctuation
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)
# tokenize
def tokenize(text):
    text = word_tokenize(text)
    return text
# remove stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    text = [i for i in text if not i in stop_words]
    return text
# lemmatize
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    text = [lemmatizer.lemmatize(token) for token in text]
    return text

def preprocessing(text):
    text = text_lowercase(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    # text = tokenize(text)
    # text = remove_stopwords(text)
    # text = lemmatize(text)
    # text = ' '.join(text) # Word2Vec wants a list of tokens
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Running preprocessing on the flattened data

In [ ]:
pp_corpus = [preprocessing(text) for text in context_string]

In [ ]:
pp_corpus[:10]

['edmund burke in the th century burke was praised by both liberals and conservatives burkes friend philip francis wrote that burke was a man who truly  prophetically foresaw all the consequences which would rise from the adoption of the french principles but because burke wrote with so much passion people were doubtful of his arguments william windham spoke from the same bench in the house of commons as burke had when he had separated from fox and an observer said windham spoke like the ghost of burke when he made a speech against peace with france in  william hazlitt a political opponent of burke regarded him as amongst his three favourite writers the others being junius and rousseau and made it a test of the sense and candour of any one belonging to the opposite party whether he allowed burke to be a great man william wordsworth was originally a supporter of the french revolution and attacked burke in a letter to the bishop of llandaff  but by the early th century he had changed his

Now I have a list of preprocessed strings

# Sentence transformer

In [ ]:
!pip install -U sentence-transformers
!pip install hnswlib

Choosing the best avaiable pre-trained model. We don't use cross embeddings because it takes time to embed the documents.

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

model = SentenceTransformer('all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
corpus_embeddings = model.encode(pp_corpus, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/4189 [00:00<?, ?it/s]

In [ ]:
corpus_embeddings.size()

torch.Size([954251, 768])

In [ ]:
corpus_embeddings[0]

tensor([-5.0160e-02,  3.7346e-02,  2.0605e-02,  3.2095e-02, -1.3090e-02,
        -3.0966e-02,  1.3471e-02, -3.3581e-02, -4.1082e-02, -5.5764e-03,
        -3.8593e-02,  4.1964e-02, -4.9155e-02, -3.3703e-02, -1.0134e-02,
        -6.6257e-03, -1.4556e-03,  8.9460e-02, -3.5593e-02, -1.2033e-02,
        -2.9109e-02, -1.5235e-02,  2.0221e-02, -1.4159e-02,  2.3192e-02,
        -2.1416e-02,  5.5793e-03,  3.4200e-02,  5.0240e-02,  4.6497e-02,
        -1.7196e-03,  1.6143e-02, -3.0520e-02, -3.6360e-02,  1.2479e-06,
         1.9724e-02, -1.9378e-02, -4.6250e-02,  5.2411e-02, -4.3808e-02,
        -2.2731e-02,  7.0532e-02, -6.4769e-02,  7.2885e-03, -1.3630e-02,
         9.1420e-02, -5.6456e-03,  1.0216e-01,  1.1434e-02,  7.6560e-02,
         4.3736e-02, -1.3981e-02,  2.3944e-02, -1.0085e-02,  1.2561e-02,
        -3.0238e-02, -3.4912e-02, -7.0260e-03, -6.6190e-02,  7.4032e-02,
        -1.0271e-02, -2.8389e-02, -5.6283e-03, -5.1427e-02, -1.9057e-03,
        -1.4767e-02,  2.9381e-02,  4.3198e-02,  1.0

In [ ]:
normlaised_docs_embeddings = util.normalize_embeddings(corpus_embeddings)

In [ ]:
# import pickle

# with open('/content/drive/MyDrive/proj-nlp-2024/data/training_docs_embeddings.pkl', 'wb') as f:
#     pickle.dump(corpus_embeddings, f)

In [ ]:
# import pickle

# with open('/content/drive/MyDrive/proj-nlp-2024/data/training_normalised_docs_embeddings.pkl', 'wb') as f:
#     pickle.dump(normlaised_docs_embeddings, f)

# Indexing

In [ ]:
import pickle

with open('/content/drive/MyDrive/proj-nlp-2024/data/training_docs_embeddings.pkl', "rb") as f:
    corpus_embeddings = pickle.load(f)

In [ ]:
!pip -q install hnswlib

In [ ]:
# import hnswlib

# index_path = '/content/drive/MyDrive/proj-nlp-2024/data/training_docs_hnswlib.index'
# index = hnswlib.Index(space='cosine', dim=768)
# index.init_index(max_elements=corpus_embeddings.size(0), ef_construction=400, M=64)
# index.add_items(corpus_embeddings.cpu(), list(range(len(corpus_embeddings))))
# index.save_index(index_path)

# Semantic Search

In [ ]:
import hnswlib

index_path = '/content/drive/MyDrive/proj-nlp-2024/data/training_docs_hnswlib.index'
index = hnswlib.Index(space='cosine', dim=768)
index.load_index(index_path)

In [ ]:
query = "They must have amnesia, they forgot that I'm him"
query_embedding = model.encode(query, convert_to_tensor=True)

In [ ]:
corpus_ids, distances = index.knn_query(query_embedding.cpu(), k=128)
scores = 1 - distances

print("Cosine similarity model search results")
print(f"Query: \"{query}\"")
print("---------------------------------------")
for idx, score in zip(corpus_ids[0], scores[0]):
    print(f"Score: {score:.4f}\nDocument: \"{context_string[idx]}\"\n\n")

Cosine similarity model search results
Query: "Why did Christ kill himself?"
---------------------------------------
Score: 0.5043
Document: "Crucifixion of Jesus John Calvin supported the "agent of God" Christology and argued that in his trial in Pilate's Court Jesus could have successfully argued for his innocence, but instead submitted to crucifixion in obedience to the Father. This Christological theme continued into the 20th century, both in the Eastern and Western Churches. In the Eastern Church Sergei Bulgakov argued that the crucifixion of Jesus was "pre-eternally" determined by the Father before the creation of the world, to redeem humanity from the disgrace caused by the fall of Adam. In the Western Church, Karl Rahner elaborated on the analogy that the blood of the Lamb of God (and the water from the side of Jesus) shed at the crucifixion had a cleansing nature, similar to baptismal water."


Score: 0.5043
Document: "Crucifixion of Jesus John Calvin supported the "agent of G

Embedding the n top results with Cross Encoder so as to actually extract an answer

In [ ]:
from sentence_transformers import CrossEncoder

xenc_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
import numpy as np

query = "What's up with airplane peanuts?"
query_embedding = model.encode(query, convert_to_tensor=True)

corpus_ids, distances = index.knn_query(query_embedding.cpu(), k=128)
scores = 1 - distances

model_inputs = [(query, context_string[idx]) for idx in corpus_ids[0]]
cross_scores = xenc_model.predict(model_inputs)

print("Cross-encoder model re-ranking results")
print(f"Query: \"{query}\"")
print("---------------------------------------")
for idx in np.argsort(-cross_scores)[:3]:
    print(f"Score: {cross_scores[idx]:.4f}\nDocument: \"{context_string[corpus_ids[0][idx]]}\"\n\n")

Cross-encoder model re-ranking results
Query: "What's up with airplane peanuts?"
---------------------------------------
Score: -1.0524
Document: "The Peanuts Movie The Peanuts Movie (known in some countries as Snoopy and Charlie Brown: A Peanuts Movie) is a 2015 American computer-animated comedy film based on Charles M. Schulz's comic strip "Peanuts", produced by Blue Sky Studios and distributed by 20th Century Fox. It is the fifth full-length "Peanuts" film, and the first in 35 years. The film is directed by Steve Martino from a screenplay by Craig and Bryan Schulz (Schulz's son and grandson, respectively), and Cornelius Uliano, and stars the voices of Noah Schnapp as Charlie Brown and, via archival recordings, Bill Melendez as Snoopy and Woodstock. The film sees Charlie Brown trying to improve his odds with the Little Red-Haired Girl, while Snoopy writes a book about the World War I Flying Ace as he imagines himself as a legend trying to save his love interest and fellow pilot Fifi 

# TSNE

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=3, n_iter=500)
tsne_embedding = tsne.fit_transform(corpus_embeddings)

In [ ]:
!pip install plotly

In [ ]:
import numpy as np
import plotly.express as px

x, y, z = np.transpose(tsne_embedding)

fig = px.scatter_3d(x=x, y=y, z=z)
fig.update_traces(marker=dict(size=3,line=dict(width=2)))
fig.show()

In [ ]:
# Select ranomly the sentences
embed_idxs = np.random.choice(np.arange(len(tsne_embedding)), 10)
sent_idxs = np.arange(0, len(sentences), 300)[embed_idxs]

# Plot the selected sentences
fig = px.scatter_3d(x=x[embed_idxs], y=y[embed_idxs], z=z[embed_idxs], text=embed_idxs)
fig.update_traces(marker=dict(size=3,line=dict(width=2)))
fig.show()

In [ ]:
for e_idx, s_idx in zip(embed_idxs, sent_idxs):
    print(f"{e_idx:6d}:\t\"{sentences[s_idx]}\"")